**извлечение признаков из текста на естественном языке**

частотный анализ, Term Frequency (TF)

_Евгений Борисов <esborisov@sevsu.ru>_

## библиотеки

In [33]:
import numpy as np
import pandas as pd

pd.options.display.max_colwidth = 200 

# вывод на печать чисел до 2 знака
pd.options.display.precision = 2 
np.set_printoptions(precision=2) 

## тексты

In [34]:
docs = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
 ]

In [36]:
def tf_table(x,v,d=docs):
    print( 'словарь:', len(v), 'слов\n') 
    return pd.concat([ 
        pd.DataFrame(x,columns=v), 
        pd.Series(d,name='Sentence')
    ],axis=1).set_index('Sentence')

## простая векторизация ( CountVectorizer )

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
tf_model = CountVectorizer().fit(docs)
 
tf_table(
    x = tf_model.transform(docs).todense(),
    v = sorted( tf_model.vocabulary_.keys() ),
    d=docs
)

словарь: 9 слов



,and,document,first,is,one,second,the,third,this
Sentence,,,,,,,,,
This is the first document.,0,1,1,1,0,0,1,0,1
This document is the second document.,0,2,0,1,0,1,1,0,1
And this is the third one.,1,0,0,1,1,0,1,1,1
Is this the first document?,0,1,1,1,0,0,1,0,1


## бинарная векторизация ( CountVectorizer )

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

tf_model = CountVectorizer(binary=True).fit(docs)

tf_table(
    x = tf_model.transform(docs).todense(),
    v = sorted( tf_model.vocabulary_.keys() ),
    d=docs
)

словарь: 9 слов



,and,document,first,is,one,second,the,third,this
Sentence,,,,,,,,,
This is the first document.,0,1,1,1,0,0,1,0,1
This document is the second document.,0,1,0,1,0,1,1,0,1
And this is the third one.,1,0,0,1,1,0,1,1,1
Is this the first document?,0,1,1,1,0,0,1,0,1


## векторизация словосочетаний ( CountVectorizer )

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

tf_model = CountVectorizer(ngram_range=(2,2)).fit(docs)

tf_table(
    x = tf_model.transform(docs).todense(),
    v = sorted( tf_model.vocabulary_.keys() ),
    d=docs
)

словарь: 13 слов



,and this,document is,first document,is the,is this,second document,the first,the second,the third,third one,this document,this is,this the
Sentence,,,,,,,,,,,,,
This is the first document.,0,0,1,1,0,0,1,0,0,0,0,1,0
This document is the second document.,0,1,0,1,0,1,0,1,0,0,1,0,0
And this is the third one.,1,0,0,1,0,0,0,0,1,1,0,1,0
Is this the first document?,0,0,1,0,1,0,1,0,0,0,0,0,1


##  CountVectorizer + TF

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
tf_model = CountVectorizer().fit(docs)

from sklearn.feature_extraction.text import TfidfTransformer
# модель обратной частоты для понижения значимости слишком частых слов
idf_model = TfidfTransformer(norm='l2').fit( tf_model.transform( docs ) )

tf_table(
    x = idf_model.transform( tf_model.transform(docs) ).todense(),
    v = sorted( tf_model.vocabulary_.keys() ),
    d=docs
)

словарь: 9 слов



,and,document,first,is,one,second,the,third,this
Sentence,,,,,,,,,
This is the first document.,0.00,0.47,0.58,0.38,0.00,0.00,0.38,0.00,0.38
This document is the second document.,0.00,0.69,0.00,0.28,0.00,0.54,0.28,0.00,0.28
And this is the third one.,0.51,0.00,0.00,0.27,0.51,0.00,0.27,0.51,0.27
Is this the first document?,0.00,0.47,0.58,0.38,0.00,0.00,0.38,0.00,0.38


### TfidfVectorizer

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

# "всё в одном" : TfidfVectorizer = CountVectorizer + TfidfTransformer
tf_model = TfidfVectorizer( use_idf=True, norm='l2').fit( docs )

tf_table(
    x = tf_model.transform(docs).todense(),
    v = sorted( tf_model.vocabulary_.keys() ),
    d=docs
)

словарь: 9 слов



,and,document,first,is,one,second,the,third,this
Sentence,,,,,,,,,
This is the first document.,0.00,0.47,0.58,0.38,0.00,0.00,0.38,0.00,0.38
This document is the second document.,0.00,0.69,0.00,0.28,0.00,0.54,0.28,0.00,0.28
And this is the third one.,0.51,0.00,0.00,0.27,0.51,0.00,0.27,0.51,0.27
Is this the first document?,0.00,0.47,0.58,0.38,0.00,0.00,0.38,0.00,0.38
